This notebook has the purpose of comparing tidal ellipses calculated by the model to other data sources.

#Loading and Preparing the Files

The following cells in this section will load, mask, rotate and unstagger the model output data. 

In [1]:
import os
import glob
import datetime
import matplotlib.pylab as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex

import netCDF4 as nc
from scipy.optimize import curve_fit
from salishsea_tools import (viz_tools,tidetools, nc_tools)
from salishsea_tools.nowcast import (research_VENUS, analyze)

%matplotlib inline

In [2]:
grid_B = nc.Dataset('/data/dlatorne/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')
lats = grid_B.variables['nav_lat'][:]
lons = grid_B.variables['nav_lon'][:]
bathy = grid_B.variables['Bathymetry'][:,:]

In [30]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'
to=datetime.datetime(2015,5,20)
tf=datetime.datetime(2015,5,28)
filesu=analyze.get_filenames(to,tf, '1h', 'grid_U', path)
filesv=analyze.get_filenames(to,tf,'1h', 'grid_V', path)

The locations of the nodes are stored in a dictionary in research_VENUS.py

In [31]:
sites=research_VENUS.SITES
i_c=sites['VENUS']['Central']['i']
i_e=sites['VENUS']['East']['i']
j_c=sites['VENUS']['Central']['j']
j_e=sites['VENUS']['East']['j']

In [36]:
u_u_c, time = analyze.combine_files(filesu, 'vozocrtx','None',[j_c-1, j_c], [i_c-1,i_c])
v_v_c, timec = analyze.combine_files(filesv, 'vomecrty','None',[j_c-1, j_c], [i_c-1,i_c])
time_c = tidetools.convert_to_seconds(timec)
dep_t_c = nc.Dataset(filesv[-1]).variables['depthv']

u_u_e, time = analyze.combine_files(filesu, 'vozocrtx','None',[j_e-1, j_e], [i_e-1,i_e])
v_v_e, timee = analyze.combine_files(filesv, 'vomecrty','None',[j_e-1, j_e], [i_e-1,i_e])
time_e = tidetools.convert_to_seconds(timee)
dep_t_e = nc.Dataset(filesv[-1]).variables['depthv']

In [65]:
u_u_0 = np.ma.masked_values(u_u_e, 0)
v_v_0 = np.ma.masked_values(v_v_e, 0)
u_u_0c = np.ma.masked_values(u_u_c, 0)
v_v_0c = np.ma.masked_values(v_v_c, 0)

u_c_all, v_c_all=research_VENUS.unstag_rot_gridded(u_u_0c, v_v_0c, 'Central')
u_e_all, v_e_all=research_VENUS.unstag_rot_gridded(u_u_0, v_v_0, 'East')

Slice in order to compare, the min and max depth values for each location is determine by the comparison source.

In [122]:
central_min = 35
central_max = 290

east_min = 20
east_max = 160

minc = [n for n,i in enumerate(dep_t_c) if i>central_min ][0]
maxc = [n for n,i in enumerate(dep_t_c) if i>central_max ][0]
mine = [n for n,i in enumerate(dep_t_e) if i>east_min ][0]
maxe = [n for n,i in enumerate(dep_t_e) if i>east_max ][0]        

u_c = u_c_all[:, minc:maxc]
v_c = v_c_all[:, minc:maxc]
u_e = u_e_all[:, mine:maxe]
v_e = v_e_all[:, mine:maxe]
u_c.shape

(216, 11)

In [101]:
uc=analyze.depth_average(u_c,dep_t_c[minc:maxc],1)
vc=analyze.depth_average(v_c,dep_t_c[minc:maxc],1)
ue=analyze.depth_average(u_e,dep_t_e[mine:maxe],1)
ve=analyze.depth_average(v_e,dep_t_e[mine:maxe],1)

In [125]:
uc_av=np.mean(uc)
vc_av=np.mean(vc)
ue_av=np.mean(ue)
ve_av=np.mean(ve)

#Fits

The following cells will calculate the cell-centered model output velocities' tidal parameters.

In [129]:
times=[time_c,time_e]
us=[uc, ue]
vs=[vc, ve]
i=np.arange(0,2)

thesize=(1,2)
vM2amp = np.zeros(thesize); vM2pha = np.zeros(thesize)
vK1amp = np.zeros(thesize); vK1pha = np.zeros(thesize)
uM2amp = np.zeros(thesize); uM2pha = np.zeros(thesize)
uK1amp = np.zeros(thesize); uK1pha = np.zeros(thesize)

for i, u, time, v in zip(i, us, times, vs):
    uM2amp[:,i], uM2pha[:,i], uK1amp[:,i], uK1pha[:,i] = tidetools.fittit(u, time)
    vM2amp[:,i], vM2pha[:,i], vK1amp[:,i], vK1pha[:,i] = tidetools.fittit(v, time) 

CX, SX, CY, SY, ap, am, ep, em, major, minor, theta = tidetools.ellipse_params (uM2amp, uM2pha, vM2amp, vM2pha)
CX_k, SX_k, CY_k, SY_k, ap_k, am_k, ep_k, em_k, major_k, minor_k, theta_k = tidetools.ellipse_params (uK1amp, uK1pha, vK1amp, vK1pha)


In [141]:
major[0,]

array([ 0.14299807,  0.25419431])

#Comparisons
##Depth Averaged at VENUS nodes 

In [143]:
majc = 0.179
minorc = 0.05
incc = 109

#CODAR values
majc_cod = 0.113
minorc_cod = 0.08
incc_cod = 127

maje = 0.262
minore = -0.04
ince = 124

maje_cod = 0.138
minore_cod = -0.037
ince_cod = 133

In [155]:
print "Central   Major-Axis   Minor-Axis     Angle"
print "           (m/s)      (m/s)     (deg ccw E)"  
print ' Model      {0:.2f}       {1:.2f}      {2:.0f}   '.format( 
         major[0,0], minor[0,0], theta[0,0]) 
print ' Nodes      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
         majc, minorc, incc)
print ' CODAR      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
         majc_cod, minorc_cod, incc_cod)


print "East   Major-Axis   Minor-Axis     Angle"
print "           (m/s)      (m/s)     (deg ccw E)"  
print ' Model      {0:.2f}       {1:.2f}      {2:.0f}   '.format( 
         major[0,1], minor[0,1], theta[0,1]) 
print ' Nodes      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
         maje, minore, ince)
print ' CODAR      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
         maje_cod, minore_cod, ince_cod)


Central   Major-Axis   Minor-Axis     Angle
           (m/s)      (m/s)     (deg ccw E)
 Model      0.14       0.02      131   
 Nodes      0.18       0.05      109   
 CODAR      0.11       0.08      127   
East   Major-Axis   Minor-Axis     Angle
           (m/s)      (m/s)     (deg ccw E)
 Model      0.25       -0.02      137   
 Nodes      0.26       -0.04      124   
 CODAR      0.14       -0.04      133   
